# Data retrieval from MySQL database

In [ ]:
# pip install mysql-connector-python

In [ ]:
# pip install pandas

In [ ]:
import mysql.connector
import pandas as pd

In [ ]:
class mysqlconnector:
    
    def __init__(self, database = 'book'):
        try:
            connection = mysql.connector.connect(host = 'localhost',
                                                user = 'root',
                                                password = '',
                                                use_pure = True,
                                                database = database
                                                )
            if connection.is_connected:
                db_info = connection.get_server_info()
                print('connected to MYSQL server version', db_info)
                print('You are connected to the database:', database)
                self.connection = connection
        except Exception as e:
            print('Error while connecting to MYSQL', e)
            
    def execute(self, query, header = False):
        cursor = self.connection.cursor(buffered = True)
        cursor.execute(query)
        
        try:
            record = cursor.fetchall()
            
            if header:
                header = [i[0] for i in cursor.description]
                return {'header': header, 'record': record}
            else:
                return record
        except:
            pass
        
        
    def to_db(self, query):
        result = self.execute(query, header = True)
        df = pd.DataFrame(result['record'])
        df.columns = result['header']
        return df

In [ ]:
db = mysqlconnector('book')

In [ ]:
query = """
        SELECT * FROM users
        """

df1 = db.to_db(query)
df1.sample(5)

In [ ]:
query = """
        SELECT * FROM books
        """

df2 = db.to_db(query)
df2.sample(5)

In [ ]:
query = """
        SELECT * FROM ratings
        """

df3 = db.to_db(query)
df3.sample(5)

In [ ]:
print(df1.shape, df2.shape, df3.shape)

# Data cleaning and transformation

In [ ]:
df2.dtypes

### YOP cleaning

In [ ]:
import numpy as np

np.sort(df2['YOP'].unique())

In [ ]:
# To diplay full text in the column of the dataframe
pd.set_option('display.max_colwidth', None)

In [ ]:
df2[df2['book_id'] == '078946697X']

In [ ]:
df2[df2['book_id'] == '2070426769']

In [ ]:
# Since the dataset was published on 2004 hence all the YOP above 2006(with two years of margin) can be considered as the error

import numpy as np

df2.loc[(df2['YOP'] > 2006) | (df2['YOP'] == 0), 'YOP'] = np.NaN

In [ ]:
df2['YOP'].isna().value_counts()

### Publisher cleaning

In [ ]:
df2['publisher'].isna().value_counts()

In [ ]:
df2[df2['publisher'] == 'N/A']

In [ ]:
# Checking clues from title and author for 'N/A' values

df2[df2['title'] == 'Tyrant Moon']

In [ ]:
df2[df2['author'] == 'Elaine Corvidae']

In [ ]:
df2[df2['title'] == 'Finders Keepers']

In [ ]:
df2[df2['author'] == 'Linnea Sinclair']

### Users cleaning

In [ ]:
df1.head()

In [ ]:
df1.dtypes

In [ ]:
df1.shape

In [ ]:
df1['age'].isna().value_counts()

In [ ]:
np.sort(df1['age'].unique())

In [ ]:
# Age above 90 and below seems to be erroneous

df1[(df1['age'] > 90) | (df1['age'] < 5)].shape

### Ratings cleaning and transformation

In [ ]:
df3.head()

In [ ]:
df3.shape

In [ ]:
df3['book_rating'].unique()

In [ ]:
# matching user_id from users to ratings dataset

df3[df3['user_id'].isin(df1['user_id'])].shape

In [ ]:
# matching book_id from books to rating dataset

df3[df3['book_id'].isin(df2['book_id'])].shape

In [ ]:
df4 = df3[df3['book_id'].isin(df2['book_id'])]

df4.head()

In [ ]:
print('rating matrix should have {} entries'.format(df1.shape[0]*df2.shape[0]),'\n',
      'however, df4 have {} entries'.format(df4.shape[0]),'\n',
      'sparsity = {} %'.format(100 - df4.shape[0]*100/(df1.shape[0]*df2.shape[0])))

In [ ]:
# book_ratings with 0 values are implicit rating and 1-10 are explicit ratings

df4_implicit = df4[df4['book_rating'] == 0]
df4_explicit = df4[df4['book_rating'] != 0]

In [ ]:
print(df4_implicit.shape, df4_explicit.shape)

In [ ]:
# pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

df4_explicit['book_rating'].hist(bins = 10)

In [ ]:
# pip install seaborn

In [ ]:
import seaborn as sns

sns.countplot(data = df4_explicit, x = 'book_rating')

# Popularity based recommendation

In [ ]:
pop_rat = pd.DataFrame(df4_explicit.groupby('book_id')['book_rating'].sum())

In [ ]:
pop_10_books = pop_rat.sort_values('book_rating', ascending = False)[:10]
pop_10_books.head()

In [ ]:
pd.merge(pop_10_books, df2, how = 'inner', on = 'book_id')

# Collaborative Filtering based recommendation

Making user-book rating matrix (using only explicit ratings) with reduced data
- Only those users who have given ratings to 100 or more books
- Only those books which has recieved aggregated 100 or more ratings

In [ ]:
df4_explicit.head()

In [ ]:
count1 = df4_explicit['user_id'].value_counts()

count2 = df4_explicit['book_id'].value_counts()

In [ ]:
user100 = count1[count1 >= 100].index
print(len(user100))

# books which have recieved atleast 100 ratings in count

book100 = count2[count2 >= 100].index
print(len(book100))

In [ ]:
# All books in df4_explicit
# df4_explicit_100 = df4_explicit[df4_explicit['user_id'].isin(user100)]

# books which have recieved atleast 100 ratings in count
df4_explicit_100 = df4_explicit[(df4_explicit['user_id'].isin(user100)) & (df4_explicit['book_id'].isin(book100))]

# books with atleast 100 ratings in aggreagate
# book100_agg = pop_rat[pop_rat['book_rating']>100].index
# print(len(book100_agg))

# df4_explicit_100 = df4_explicit[(df4_explicit['user_id'].isin(user100)) & (df4_explicit['book_id'].isin(book100_agg))]

df4_explicit_100.shape

In [ ]:
# Making user-book rating matrix

rating_matrix = df4_explicit_100.pivot(index = 'user_id', columns = 'book_id', values = 'book_rating')

In [ ]:
rating_matrix

In [ ]:
rating_matrix.fillna(0, inplace = True)

In [ ]:
rating_matrix.head()

In [ ]:
rating_matrix = rating_matrix.astype('int32')

rating_matrix.head()

In [ ]:
print('sparsity of the rating matrix is {} %'.format((1-(rating_matrix.sum().sum())/rating_matrix.size)*100))

In [ ]:
# pip install sklearn

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
def similar_users(user_id, ratings):
    
#     user_ind = ratings.index.get_loc(user_id)
    NN = NearestNeighbors()
    NN.fit(ratings)
    distances, indices = NN.kneighbors(ratings.loc[user_id, :].values.reshape(1, -1), n_neighbors = 10)
    
    similarities = 1 - distances.flatten()
    return(similarities.flatten(), indices.flatten())

In [ ]:
similar_users(2033, rating_matrix)

In [ ]:
def predict_rating(user_id, item_id, ratings):
    
    similarities, indices = similar_users(user_id, ratings)
    
    wtd_sum = 0
    
    item_ind = rating_matrix.columns.get_loc(item_id)
    user_ind = rating_matrix.index.get_loc(user_id)
    
    for i in range(len(indices)):
        if indices[i] == user_ind:
            continue
        else:
            wtd_sum = wtd_sum + ((ratings.iloc[indices[i], item_ind] - rating_matrix.iloc[indices[i]].mean())*similarities[i])
    
    wtd_sum = wtd_sum + rating_matrix.iloc[user_ind].mean()
    predicted_rating = wtd_sum/(similarities.sum() - 1)
    
    if predicted_rating < 0:
        predicted_rating = 0
    elif predicted_rating > 10:
        predicted_rating = 10
        
#     print(predicted_rating)
    return(predicted_rating)

In [ ]:
predict_rating(2033, '0060392452', rating_matrix)

## To check predicted rating for specific user_id and book_id

In [ ]:
NN = NearestNeighbors()
NN.fit(rating_matrix)
distances, indices = NN.kneighbors(rating_matrix.loc[2033, :].values.reshape(1, -1), n_neighbors = 10)
    
similarities = 1 - distances.flatten()
indices = indices.flatten()

wtd_sum = 0
    
item_ind = rating_matrix.columns.get_loc('043935806X')
user_ind = rating_matrix.index.get_loc(2033)    
    
for i in range(len(indices)):
    if indices[i] == user_ind:
        continue
    else:
        wtd_sum = wtd_sum + ((rating_matrix.iloc[indices[i], item_ind] - rating_matrix.iloc[indices[i]].mean())*similarities[i])

wtd_sum = rating_matrix.iloc[user_ind].mean() + wtd_sum
predicted_rating = wtd_sum/(similarities.sum() - 1)

if predicted_rating < 0:
    predicted_rating = 0
elif predicted_rating > 10:
    predicted_rating = 10
        
print(predicted_rating)

## To check predicted rating of specific user_id and all book_id in the rating_matrix

In [ ]:
NN = NearestNeighbors()
NN.fit(rating_matrix)
distances, indices = NN.kneighbors(rating_matrix.loc[2033, :].values.reshape(1, -1), n_neighbors = 10)
    
similarities = 1 - distances.flatten()
indices = indices.flatten()

user_ind = rating_matrix.index.get_loc(2033)

for j in rating_matrix.columns:
    wtd_sum = 0
    item_ind = rating_matrix.columns.get_loc(j)
    for i in range(len(indices)):
        if indices[i] == user_ind:
            continue
        else:
#             wtd_sum = wtd_sum + ((rating_matrix.iloc[indices[i], item_ind])*similarities[i])

#             when optimistic and passimistic users influence on predicted rating is considered
            wtd_sum = wtd_sum + ((rating_matrix.iloc[indices[i], item_ind] - rating_matrix.iloc[indices[i]].mean())*similarities[i])
    
    wtd_sum = rating_matrix.iloc[user_ind].mean() + wtd_sum
    predicted_rating = wtd_sum/(similarities.sum() - 1)

    if predicted_rating < 0:
        predicted_rating = 0
    elif predicted_rating > 10:
        predicted_rating = 10
        
    print(predicted_rating, item_ind)

In [ ]:
# to ignore userwarnings

import warnings
warnings.filterwarnings('ignore')

## To print user_id, top 10 predicted rating and book_id for all users

In [ ]:
# def topn_recommendation(user_id, rating_matrix):
    
#     rating_prediction = []
#     for i in rating_matrix.columns:
#         rating_prediction.append(predict_rating(user_id, i,  rating_matrix))
#     return(rating_prediction)

# for j in rating_matrix.index:
#     x = topn_recommendation(j, rating_matrix)
#     y = np.argsort(x)[::-1][:10]
    
#     print(j, '\n', sorted(x, reverse = True)[:10], '\n', rating_matrix.columns[y].values, '\n\n')

In [ ]:
def topn_recommendation(user_id, rating_matrix):
    
    predicted_rating_all_items = []
    for i in rating_matrix.columns:
        predicted_rating_all_items.append(predict_rating(user_id, i,  rating_matrix))
   
#     predicted_rating_top10 = sorted(predicted_rating_all_items, reverse = True)[:10]
    top_10_recomm_index = np.argsort(predicted_rating_all_items)[::-1][:10]
    top_10_recomm_bookid = rating_matrix.columns[top_10_recomm_index].values
    top_10_recomm_title = df2[df2['book_id'].isin(top_10_recomm_bookid.tolist())]['title'].values
    
    return(top_10_recomm_title.tolist())

In [ ]:
topn_recommendation(2276, rating_matrix)

# Collaborative Filtering with LightFM

In [ ]:
count1 = df4_explicit['user_id'].value_counts()

count2 = df4_explicit['book_id'].value_counts()


user20 = count1[count1 >= 20].index
print(len(user20))

# books which have recieved atleast 20 ratings in count

book20 = count2[count2 >= 20].index
print(len(book20))



# books which have recieved atleast 100 ratings in count

df4_explicit_20 = df4_explicit[(df4_explicit['user_id'].isin(user20)) & (df4_explicit['book_id'].isin(book20))]

df4_explicit_20.head()

# Making user-book rating matrix

# rating_matrix = df4_explicit_20.pivot(index = 'user_id', columns = 'book_id', values = 'book_rating')

# rating_matrix.fillna(0, inplace = True)

# rating_matrix = rating_matrix.astype('int32')

# rating_matrix.head()

# print('sparsity of the rating matrix is {} %'.format((1-(rating_matrix.sum().sum())/rating_matrix.size)*100))



In [ ]:
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import coo_matrix

In [ ]:
# Function to give train and test coordinate matrices and train matrix in raw form (df3 form)

def train_test_rawtrain(df4_explicit_20, train_ratio):
    
    rows_in_split1 = int(df4_explicit_20.shape[0]*train_ratio)
    
    df_train = df4_explicit_20.iloc[:rows_in_split1]
    df_test = df4_explicit_20.iloc[rows_in_split1:]
    
    df_test = df_test[(df_test['user_id'].isin(set(df_train['user_id']))) & (df_test['book_id'].isin(set(df_train['book_id'])))]
    
    
    df_train_trans = dict()
    df_test_trans = dict()
    
    for i in ['user_id', 'book_id']:
        cat_encode = LabelEncoder()
        df_train_trans[i] = cat_encode.fit_transform(df_train[i].values)
        df_test_trans[i] = cat_encode.transform(df_test[i].values)
      
    
    ratings = dict()
    cat_encode = LabelEncoder()
    ratings['train'] = cat_encode.fit_transform(df_train['book_rating'])
    ratings['test'] = cat_encode.transform(df_test['book_rating'])
    
    n_users = len(set(df_train_trans['user_id']))
    n_books = len(set(df_train_trans['book_id']))
    
    train = coo_matrix((ratings['train'], (df_train_trans['user_id'], df_train_trans['book_id'])), shape = (n_users, n_books))
    test = coo_matrix((ratings['test'], (df_test_trans['user_id'], df_test_trans['book_id'])), shape = (n_users, n_books))      
    
    return(train, test, df_train)   

In [ ]:
train, test, train_raw = train_test_rawtrain(df4_explicit_20, 0.70)

In [ ]:
# conda install -c conda-forge lightfm

In [ ]:
from lightfm import LightFM

In [ ]:
lfm = LightFM(no_components = 100, learning_rate = 0.025, loss = 'warp')

In [ ]:
lfm.fit(train, epochs = 10, num_threads = 4)

In [ ]:
from lightfm.evaluation import auc_score

In [ ]:
auc_score(lfm, train).mean()

In [ ]:
auc_score(lfm, test).mean()

In [ ]:
train_val = train_raw.pivot(index = 'user_id', columns = 'book_id', values = 'book_rating')

In [ ]:
train_val.fillna(0, inplace = True)
train_val = train_val.astype('int')

In [ ]:
train_val.head()

In [ ]:
train_val.shape

In [ ]:
# Function to give user ({user_id : counter}) and book ({book_id : title}) dictionary

def user_book_dict(train_val, df2):
    
    user_ids = list(train_val.index)
    user_dict = dict()
    c = 0
    
    for i in user_ids:
        user_dict[i] = c
        c = c+1
        
    book_dict = dict()
    for i in range(df2.shape[0]):
        book_dict[df2.loc[i, 'book_id']] = df2.loc[i, 'title']
        
#     or use below to create book dictionary
#     book_dict = dict(zip(df2['book_id'].values, df2['title'].values))
        
    return(user_dict, book_dict)

In [ ]:
user_dict, book_dict = user_book_dict(train_val, df2)

In [ ]:
# Function to give liked books and recommended books for a specific user
# threshold can be used to control the know books by user

def topm_recommendation(user_id, lfm, train_val, user_dict, book_dict, threshold = 0):
    
    n_users, n_books =  train_val.shape
    
    user = user_dict[user_id]
    
    scores = pd.Series(lfm.predict(user, np.arange(n_books)))
    scores.index = train_val.columns
    
    scores = list(scores.sort_values(ascending = False).index)
    
    known_books = list(train_val.loc[user_id][train_val.loc[user_id, :] > threshold].sort_values(ascending = False).index)
   
    scores = [i for i in scores if i not in known_books]
    
    known_books = list(pd.Series(known_books).apply(lambda x: book_dict[x]))
    scores = list(pd.Series(scores).apply(lambda x: book_dict[x]))
    
    print("Liked books by user:")
    for i in known_books[:10]:
        print(i)
        
    print("\n Recommended books:")    
    for i in scores[:10]:
        print(i)
    
    
#     return (scores)

In [ ]:
topm_recommendation(254, lfm, train_val, user_dict, book_dict)

In [ ]:
# Function to give recommended users for specific item

def users_as_per_item(train_val, lfm, book_id, n_users_for_book):
    
    n_users, n_books = train_val.shape
    book_ids = train_val.columns
    
    scores = list(lfm.predict(np.arange(n_users), np.repeat(book_ids.searchsorted(book_id), n_users)))
    users = np.argsort(scores)[::-1][:n_users_for_book]
#     np.argsort()
    
    return(users)

In [ ]:
users_as_per_item(train_val, lfm, '002542730X', 10)

In [ ]:
from scipy.sparse import csr_matrix

x = csr_matrix(lfm.item_embeddings)

In [ ]:
x.shape

In [ ]:
train.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

df_item_simil = pd.DataFrame(cosine_similarity(x))
df_item_simil.columns = df_item_simil.index = train_val.columns
df_item_simil.head()

In [ ]:
# Function to give bought-together items (similar items) for a specific item
# For better results the main dataframe (df4_explicit_20) can further be filtered with the users and books with atleast 100 ratings. 
# So the prediction will me made based on the users who have bought and rated more books
# and the books with more ratings (popular) 

def bought_together(item_similarity_matrix, item_id, book_dict):
    
    recommended_3_books = list(item_similarity_matrix.loc[item_id, :].sort_values(ascending = False).index[1:4])
    
    print('bought together with :', book_dict[item_id])
    for i in recommended_3_books:
        print(book_dict[i])
#     return(recommended_3_books)

In [ ]:
bought_together(df_item_simil, '0439139597', book_dict)

In [ ]:
list(book_dict.keys())[list(book_dict.values()).index('Harry Potter and the Goblet of Fire (Book 4)')]